# Date voorbereiden

In [15]:
#Libaries importeren
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [26]:
#Lijst maken met alle recepten zonder noten -> csv was al door Martti gemaakt
recepten_df=pd.read_csv("Generated Datasets/Filter_noten_maaltijd_DF.csv")
recepten_zonder_noten = recepten_df.iloc[:,0].tolist()

In [110]:
#Data ophalen
tags = pd.read_csv('/data/foodboost/tags.csv')
ingr = pd.read_csv('/data/foodboost/ingredients.csv')
print("#recepten:", len(ingr.axes[0]))

#recepten: 71809


## Filteren op maaltijd (lunch en diner) en noten

In [111]:
#toegestaande tags
tagsmaal= ['lunch','diner','hoofdgerecht']
#filteren op die tags en elk recept 
recepten_lunch_diner = tags[tags['tag'].str.contains('|'.join(tagsmaal))]
recepten_lunch_diner = list(recepten_lunch_diner["recipe"].unique())

#recepten zowel zonder noten als de juiste maaltijd
recepten_zonder_noten_lunch_diner = [value for value in recepten_lunch_diner if value in recepten_zonder_noten]
print("#recepten:", len(recepten_zonder_noten_lunch_diner))

#recepten: 4593


## Ingrediënten pivot tabel maken

In [112]:
#Ingrediënten van de recepten zonder noten en alleen lunch en diner
ingr = ingr[ingr['recipe'].str.contains('|'.join(recepten_zonder_noten_lunch_diner))]
print("#recepten:", len(ingr.axes[0]))

/tmp/ipykernel_30242/3269571354.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ingr = ingr[ingr['recipe'].str.contains('|'.join(recepten_zonder_noten_lunch_diner))]


#recepten: 40746


In [115]:
#Een pivot tabel maken 
ingr_pivot = ingr.pivot_table(values="recipe", index= ingr.recipe, columns="ingredient", aggfunc="count")
ingr_pivot= ingr_pivot.fillna(0) 

print("#recepten:", len(ingr_pivot.axes[0]))
print("#ingrediënten:", len(ingr_pivot.axes[1]))

#recepten: 4616
#ingrediënten: 5100


## Dezelfde ingrediënten samenvoegen als ze te veel op elkaar lijken

In [116]:
#Variabele defineren
ingr_pivot_columnnames= ingr_pivot.columns.values.tolist()
lengte= len(ingr_pivot_columnnames)
dubbele_ingr= []
grens_ratio= 85
ingr_pivot_copy= ingr_pivot.copy()

In [117]:
#Dubbele kolomnamen zoeken
for i in range(lengte):
    if i < (lengte-1):
        Ratio = fuzz.token_sort_ratio(ingr_pivot_columnnames[i],ingr_pivot_columnnames[i+1])
        if Ratio > grens_ratio:
            dubbele_ingr.append((ingr_pivot_columnnames[i], ingr_pivot_columnnames[i+1]))

In [118]:
#Dubbele kolommen samenvoegen en de extra verwijderen
ingr_pivot_copy= ingr_pivot.copy()
for i in dubbele_ingr:
    ingr_pivot_copy[i[1]] = ingr_pivot_copy[i[0]] + ingr_pivot_copy[i[1]]
    ingr_pivot_copy=ingr_pivot_copy.drop([i[0]], axis=1)
    
print("#recepten:", len(ingr_pivot_copy.axes[0]))
print("#ingrediënten:", len(ingr_pivot_copy.axes[1]))

#recepten: 4616
#ingrediënten: 4797


## filter op ingrediënten die meer dan 40x voorkomen

In [119]:
ingr_pivot_copy = ingr_pivot_copy.loc[:,ingr_pivot_copy.sum()>40]
#Alleen recepten die nog minimaal 1 ingrediënt hebben
ingr_pivot_copy = ingr_pivot_copy.loc[ingr_pivot_copy.sum(axis=1)!=0,:]

print("#recepten:", len(ingr_pivot_copy.axes[0]))
print("#ingrediënten:", len(ingr_pivot_copy.axes[1]))

#recepten: 4556
#ingrediënten: 166


# output

In [120]:
print("#recepten:", len(ingr_pivot_copy.axes[0]))
print("#ingrediënten:", len(ingr_pivot_copy.axes[1]))
ingr_pivot_copy.head(3)

#recepten: 4556
#ingrediënten: 166


ingredient,(arachide)olie,(olijf)olie,Franse mosterd,Griekse yoghurt,Japanse sojasaus,Parmezaanse kaas,Parmigiano Reggiano,Pecorino Romano (kaas),Tasty Tom trostomaat,aardappelen,...,water,winterpeen,witlof,witte kaas 45+,witte rijst,zilvervliesrijst,zoete aardappel,zoete chilisaus,zonnebloemolie,zout
recipe,,,,,,,,,,,,,,,,,,,,,
Andijviestamppot met gebakken salami,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Libanese wraps met appel-koolsalade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Plaattaart met gegrilde groenten en witte kaas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#bestand opslaan
ingr_pivot_copy.to_csv('ingr_pivot_lunch_diner_noten.csv')